In [3]:
# This notebook evaluates the actual model results

In [1]:
import numpy as np

import keras.backend as K
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical

from hyperas import optim
from hyperas.distributions import choice, uniform, conditional
from hyperopt import Trials, STATUS_OK, tpe

import os
import argparse
import json
import random

Using TensorFlow backend.


In [2]:
def data(x_value, y_value):
    def load_x_indices():
        if x_value == "fixations":
            return [0]
        elif x_value == "xy":
            return [1]
        elif x_value == "fixationsxy":
            return [0, 1]
        else:
            return "Unknown input variables"

    def load_y_index_and_classes():
        if y_value == "binary":
            return (4, 2)
        elif y_value == "binaryno23":
            return (5, 1)
        elif y_value == "linear":
            return (3, 5)
        else:
            return "Unknown predictor variable"

    # Set the seed to achieve same sorting
    random.seed(6781693213)

    ids = list(range(0, 999))
    random.shuffle(ids)

    ids_train = ids[:880]
    ids_test = ids [880:]

    x_data_indices = load_x_indices()
    y_data_index = load_y_index_and_classes()[0]

    with open("modified_data.txt") as data:
        rows = json.load(data)

        # Remove NAN values if 3 is removed
        if y_data_index == 5:
            ids_train = [x[0] for x in enumerate(rows[5][:880]) if not x[1] == None]
            ids_test = np.array([x[0] for x in enumerate(rows[5][880:]) if not x[1] == None]) + 880
        elif x_data_indices[0] == 1:
            ids_train = [x[0] for x in enumerate(rows[1][:880]) if len(x[1]) >= 213]
            ids_test = [x[0] for x in enumerate(rows[1][880:]) if len(x[1]) >= 213]

        y_train = (np.take(rows[y_data_index], ids_train)).reshape(-1, 1)
        y_test = (np.take(rows[y_data_index], ids_test)).reshape(-1, 1) 

        if len(x_data_indices) == 2:
            x_train = np.take(rows[x_data_indices[0]:x_data_indices[1]], ids_train).reshape(-1, 1, 1)
            x_test = np.take(rows[x_data_indices[0]:x_data_indices[1]], ids_test).reshape(-1, 1, 1)
        elif x_data_indices[0] == 1:
            x_train = np.array([np.array(x[:213]) for x in np.take(rows[1], ids_train)])
            x_train = np.delete(x_train, 422, axis=0)
            y_train = np.delete(y_train, 422, axis=0)
            x_train = np.array([x.reshape(213, 2) for x in x_train])
            x_test = np.array([np.array(x[:213]) for x in np.take(rows[1], ids_test)])
        else:
            x_train = np.array([np.array(x) for x in np.take(rows[x_data_indices[0]], ids_train)]).reshape(-1, 1, 1)
            x_test = np.array([np.array(x) for x in np.take(rows[x_data_indices[0]], ids_test)]).reshape(-1, 1, 1)

    return x_train, y_train, x_test, y_test

In [3]:
# Thanks to: https://github.com/keras-team/keras/issues/6507
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def recall(y_true, y_pred):
    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    return c1 / c3

In [4]:
def evaluate_model(x, y):
    filename = "model_" + x + "_" + y
    with open(filename + ".model") as d:
        model = model_from_json(json.load(d))
        model.load_weights(filename + ".weights")
        model.compile(loss='binary_crossentropy', metrics=['binary_accuracy', recall, f1_score], optimizer='rmsprop')
        x_train, y_train, x_test, y_test = data(x, y)
        xs = model.evaluate(x_test, y_test)
        print('metrics:', model.metrics_names)
        print('values:', xs)

In [5]:
evaluate_model("fixations", "binary")

119/119 [==============================] - 0s 452us/step
metrics: ['loss', 'binary_accuracy', 'recall', 'f1_score']
values: [0.66496439390823625, 0.63025209783505998, 1.0, 0.76901501667599714]


In [6]:
evaluate_model("fixations", "binaryno23")

98/98 [==============================] - 0s 886us/step
metrics: ['loss', 'binary_accuracy', 'recall', 'f1_score']
values: [0.67507845041703207, 0.6428571428571429, 1.0, 0.78230304499061742]


In [50]:
evaluate_model("xy", "binary")

120/120 [==============================] - 1s 8ms/step
metrics: ['loss', 'binary_accuracy', 'recall', 'f1_score']
values: [0.64706189632415767, 0.6583333373069763, 0.96103738149007156, 0.77964220841725662]


In [51]:
evaluate_model("xy", "binaryno23")

98/98 [==============================] - 1s 12ms/step
metrics: ['loss', 'binary_accuracy', 'recall', 'f1_score']
values: [5.6937091009957452, 0.6428571428571429, 1.0, 0.78230304499061742]


In [45]:
def predict_model(x, y):
    filename = "model_" + x + "_" + y
    with open(filename + ".model") as d:
        model = model_from_json(json.load(d))
        model.load_weights(filename + ".weights")
        model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer='rmsprop')
        x_train, y_train, x_test, y_test = data(x, y)
        print(np.stack([model.predict_classes(x_test), y_test], axis=2).reshape(-1, 2))

In [46]:
predict_model("fixations", "binary")

[[1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


In [47]:
predict_model("fixations", "binaryno23")

[[1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]]


In [48]:
predict_model("xy", "binary")

[[1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [0 0]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [0 0]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [0 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [0 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]]


In [49]:
predict_model("xy", "binaryno23")

[[1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [1 1]
 [1 0]
 [1 0]
 [1 0]
 [1 1]]
